<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#3.-Principal-Component-Analysis" data-toc-modified-id="3.-Principal-Component-Analysis-1">3. Principal Component Analysis</a></span></li><li><span><a href="#9.-Bayes-and-KNN-Classifier" data-toc-modified-id="9.-Bayes-and-KNN-Classifier-2">9. Bayes and KNN Classifier</a></span><ul class="toc-item"><li><span><a href="#9.1.-Bayes" data-toc-modified-id="9.1.-Bayes-2.1">9.1. Bayes</a></span></li><li><span><a href="#9.2.-KNN" data-toc-modified-id="9.2.-KNN-2.2">9.2. KNN</a></span></li></ul></li></ul></div>

# 3. Principal Component Analysis

In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

np.random.seed(503)

In [2]:
# Create data matrix
X = np.array([
    [1, 0, 0.5],
    [6, 14, 3],
    [11, 28, 5.5],
    [7, 21, 3.5]
])

In [3]:
# Standardize for PCA
X_standardized = StandardScaler().fit_transform(X)
X_standardized

array([[-1.47391105, -1.52127766, -1.47391105],
       [-0.07018624, -0.16903085, -0.07018624],
       [ 1.33353857,  1.18321596,  1.33353857],
       [ 0.21055872,  0.50709255,  0.21055872]])

In [4]:
pca = PCA(n_components=1)
pca.fit(X_standardized)

PCA(copy=True, iterated_power='auto', n_components=1, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [5]:
# First principal direction
print(pca.components_)

[[-0.57834475 -0.57535615 -0.57834475]]


In [6]:
# First principal components
print(pca.components_ @ X_standardized.T)

[[ 2.5801339   0.17843663 -2.22326064 -0.53530988]]


In [7]:
# Remaining variance
print(1 - pca.explained_variance_ratio_)

[0.00680207]


# 9. Bayes and KNN Classifier

## 9.1. Bayes

In [8]:
import numpy as np
import scipy.linalg as linalg
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

np.random.seed(503)

In [9]:
# Load data
data = np.loadtxt("data/usps-2cls.csv", delimiter=",")
data[:5, :]

array([[ 0.,  0., 95., ...,  0.,  0.,  0.],
       [ 0., 78., 95., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [10]:
# Designate feature matrix X, response vector y
X = data[:, :-1]
y = data[:, -1]

print(f"X = {X.shape} matrix:\n{X[:5, :]}\n")
print(f"y = {y.shape} vector:\n{y[:5]}")

X = (2200, 256) matrix:
[[ 0.  0. 95. ...  0.  0.  0.]
 [ 0. 78. 95. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]

y = (2200,) vector:
[0. 0. 0. 0. 0.]


In [11]:
def calculate_means(X, y):
    """Calculates mean vectors for each class c found in the response vector y."""
    mus = []
    for c in np.sort(np.unique(y)):
        mus.append(np.mean(X[(y == c), :], axis=0))
    
    return mus

In [12]:
def calculate_covs(X, y):
    """Calculates covariance matrices for each class c found in the response vector y."""
    Sigmas = []
    for c in np.sort(np.unique(y)):
        # rowvar: data are rows; bias: calculate sample cov (normalize by N-1)
        Sigmas.append(np.cov(X[(y == c), :], rowvar=False, bias=False))
    
    return Sigmas

### Some notes on simplification

The function below uses the faster (and more stable) method of calculating probabilities under a multivariate Gaussian distribution with high dimensionality. This method was documented earlier in this course and gets around numerical issues that occur when attempting to invert the covariance matrix (see `speed_up_GMM.pdf`). To recap the method:

1. Compute eigendecomposition of $\Sigma = U \Lambda U^\top$
2. Approximate $\tilde{\Sigma}$ (the low-rank approximation of $\Sigma$) $ = \tilde{U}\tilde{\Lambda}\tilde{U}$ where $\tilde{U}$ is the matrix of the $r$ eigenvectors associated with the $r$ largest eigenvalues of $\Sigma$ and $\tilde{\Lambda}$
3. Compute $\tilde{x} = \tilde{U}^\top x$, the low-rank approximation of the dataset
4. Compute $\tilde{mu} = \tilde{U}^\top \mu$, the low-rank approximation of the mean vector for the distribution

With those approximations, calculate density:

$$ \mathcal{N}(x; \mu, \Sigma) \approx \frac{1}{\sqrt{(2\pi)^d \prod_{i=1}^{r} \lambda_i}} exp\left\{ -\frac{1}{2} \sum_{i=1}^{r} \frac{(\tilde{x}_i - \tilde{\mu}_i)^2}{\lambda_i} \right\} $$

In the formula above, $det(\Sigma)$ is replaced by $det(\tilde{\Sigma}) = \prod_{i=1}^{r} \lambda_i$.

The formula can also be written as:
$$ \mathcal{N}(x; \mu, \Sigma) \approx \frac{exp\left\{ -\frac{1}{2} \sum_{i=1}^{r} \frac{(\tilde{x}_i - \tilde{\mu}_i)^2}{\lambda_i} \right\}}{\sqrt{(2\pi)^d \prod_{i=1}^{r} \lambda_i}} \approx \frac{exp\left\{ -\frac{1}{2} \sum_{i=1}^{r} \frac{(\tilde{x}_i - \tilde{\mu}_i)^2}{\lambda_i} \right\}}{\sqrt{(2\pi)^d} \cdot \sqrt{\prod_{i=1}^{r} \lambda_i}} $$

Since $\sqrt{(2\pi)^d}$ is just a constant, we can eliminate that - it does not matter for the purpose of comparing likelihoods, as the constant is the same for both distributions. This means we only need to calculate $\sqrt{\prod_{i=1}^r \lambda_i}$. Omitting the constant results in:
$$ \ell(x, \mu, \Sigma) \approx \mathcal{N}(x; \mu, \Sigma) \approx \frac{exp\left\{ -\frac{1}{2} \sum_{i=1}^{r} \frac{(\tilde{x}_i - \tilde{\mu}_i)^2}{\lambda_i} \right\}}{\sqrt{\prod_{i=1}^{r} \lambda_i}} $$

In addition, the function serves the same purpose as calculating $p(x_i|y)$ for use with Bayes' theorem: $p(y|x) = \frac{p(x|y)p(y)}{p(x)}$.

However, we only need the $p(x|y)$ part of Bayes' theorem for this problem because:
1. $p(x)$ is constant between the two classes (as a marginal probability)
2. $p(y = 0) = p(y = 1) = 0.5$

In addition, as noted above, the actual calculation used in the function below will not return the true probability, because the true probability is not needed for selecting a class - only relative likelihood.

In [13]:
def calculate_likelihood(X, mu, Sigma, r=None):
    """Calculates likelihood under a multivariate normal distribution.
    
    Some constant/scaling terms are left out for simplicity, so this is not true probability, but it serves the purpose
    of determining relative p(x|y) given a class y, a mean vector mu of features for data points in class y, and a covariance
    matrix Sigma. This assumes a Gaussian/normal distribution of data for each feature.
    
    Parameters
    ----------
    X : numpy array
        A data matrix in row format (features as column vectors) with shape (m, n).
    Sigma : numpy array
        A covariance matrix with shape (n, n).
    mu : numpy array
        A mean vector with shape (n, ).
    r : int
        A rank r for approximating Sigma in order to efficiently calculate probabilities under a Gaussian
        distribution using the sped-up method.
    
    Returns
    -------
    density : numpy array
        An array of probabilities with shape (m, ).       
    """
    m, n = X.shape
    
    if r is None:
        r = n
    
    # Eigendecomposition of Sigma, taking only the top r eigenpairs
    lam_approx, U_approx = linalg.eigh(Sigma, eigvals=(n-r, n-1))
    
    # Approximate the data and means
    X_approx = (U_approx.T @ X.T).T  # Approximation of the input data
    mu_approx = U_approx.T @ mu.reshape(-1, 1)
    
    # Numerator of density function (passed to exp as needed)
    normalized_distances_sq = -0.5 * np.sum(((X_approx - mu_approx.T)**2) / lam_approx, axis=1)
    
    # Denominator of density function with constant removed
    lam_product_sq = np.sqrt(np.product(lam_approx))
    
    # Calculate density using rank-r approximations
    density = np.exp(normalized_distances_sq) / lam_product_sq
    
    return density

In [14]:
def calculate_misclassification(X, y, mus, Sigmas, r):
    """Calculates misclassification rate given multiple a list of mean vectors (mus) and covariance matrices (Sigmas)."""
    
    # Calculate likelihood under each class' multivariate Gaussian distribution
    likelihoods = []
    for mu, Sigma in zip(mus, Sigmas):
        likelihoods.append(calculate_likelihood(X, mu, Sigma, r=r))
    
    # Pick a class: under which distribution does x_i have a greater likelihood?
    likelihoods = np.array(likelihoods).T    
    predicted_y = np.argmax(likelihoods, axis=1)
    
    # Compare selected classes to actual labels
    error = np.where(y != predicted_y)[0].shape[0] / y.shape[0]
    
    return likelihoods, predicted_y, error

In [15]:
training_errors = {}
test_errors = {}

for p in [0.1, 0.2, 0.5, 0.8, 0.9]:
    print(f"Evaluating p={p}")
    training_error = []
    test_error = []
    for iteration in range(100):        
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=p)
        mus = calculate_means(X_train, y_train)  # Estimate mean vectors from training data
        Sigmas = calculate_covs(X_train, y_train)   # Estimate covariance matrices from training data

        # Calculate misclassification using learned parameters
        # r=50 was determined empirically to be a good value (from my experience with Homework 4)
        training_results = calculate_misclassification(X_train, y_train, mus, Sigmas, r=50)
        test_results = calculate_misclassification(X_test, y_test, mus, Sigmas, r=50)
        
        training_error.append(training_results[2])
        test_error.append(test_results[2])
        
    training_errors[p] = np.mean(training_error)
    test_errors[p] = np.mean(test_error)

Evaluating p=0.1
Evaluating p=0.2
Evaluating p=0.5
Evaluating p=0.8
Evaluating p=0.9


In [16]:
import pprint

print("Training errors:")
pprint.pprint(training_errors)
print("\nTest errors:")
pprint.pprint(test_errors)

Training errors:
{0.1: 0.09836363636363638,
 0.2: 0.09425000000000003,
 0.5: 0.0753909090909091,
 0.8: 0.06661363636363637,
 0.9: 0.06587878787878787}

Test errors:
{0.1: 0.12046969696969696,
 0.2: 0.0872784090909091,
 0.5: 0.0689181818181818,
 0.8: 0.06409090909090909,
 0.9: 0.061090909090909085}


## 9.2. KNN

In [17]:
from sklearn.neighbors import KNeighborsClassifier

np.random.seed(503)

In [18]:
training_errors = {}
test_errors = {}

for K in [5, 10, 15, 30]:
    print(f"Evaluating K={K}")
    training_errors[K] = {}
    test_errors[K] = {}
    for p in [0.1, 0.2, 0.5, 0.8, 0.9]:
        print(f"  Evaluating p={p}")
        training_error = []
        test_error = []
        for i in range(100):
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=p)
            
            knn = KNeighborsClassifier(n_neighbors=K)
            knn.fit(X_train, y_train)
            
            training_predictions = knn.predict(X_train)
            training_error.append(np.where(y_train != training_predictions)[0].shape[0] / y_train.shape[0])
            
            test_predictions = knn.predict(X_test)
            test_error.append(np.where(y_test != test_predictions)[0].shape[0] / y_test.shape[0])
            
        training_errors[K][p] = np.mean(training_error)
        test_errors[K][p] = np.mean(test_error)

Evaluating K=5
  Evaluating p=0.1
  Evaluating p=0.2
  Evaluating p=0.5
  Evaluating p=0.8
  Evaluating p=0.9
Evaluating K=10
  Evaluating p=0.1
  Evaluating p=0.2
  Evaluating p=0.5
  Evaluating p=0.8
  Evaluating p=0.9
Evaluating K=15
  Evaluating p=0.1
  Evaluating p=0.2
  Evaluating p=0.5
  Evaluating p=0.8
  Evaluating p=0.9
Evaluating K=30
  Evaluating p=0.1
  Evaluating p=0.2
  Evaluating p=0.5
  Evaluating p=0.8
  Evaluating p=0.9


In [19]:
print("Training errors:")
pprint.pprint(training_errors)
print("\nTest errors:")
pprint.pprint(test_errors)

Training errors:
{5: {0.1: 0.04945454545454546,
     0.2: 0.03368181818181818,
     0.5: 0.021445454545454556,
     0.8: 0.017954545454545452,
     0.9: 0.0171010101010101},
 10: {0.1: 0.09731818181818182,
      0.2: 0.06529545454545455,
      0.5: 0.038172727272727275,
      0.8: 0.029431818181818184,
      0.9: 0.028424242424242428},
 15: {0.1: 0.09313636363636364,
      0.2: 0.0613409090909091,
      0.5: 0.04017272727272728,
      0.8: 0.029255681818181816,
      0.9: 0.028106060606060614},
 30: {0.1: 0.1390909090909091,
      0.2: 0.10147727272727272,
      0.5: 0.062272727272727264,
      0.8: 0.04784090909090911,
      0.9: 0.04576262626262626}}

Test errors:
{5: {0.1: 0.09110101010101011,
     0.2: 0.06082954545454545,
     0.5: 0.03566363636363636,
     0.8: 0.031818181818181815,
     0.9: 0.02959090909090909},
 10: {0.1: 0.12991919191919193,
      0.2: 0.08725000000000001,
      0.5: 0.050245454545454545,
      0.8: 0.036000000000000004,
      0.9: 0.03504545454545455},
 15: 